# Austrian energy system Tutorial Part 4: Investigating Many Policy Scenarios

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Austrian energy system baseline scenario (`austria.ipynb`) and solved it successfully
- You have completed the tutorial on introducing one policy scenario (`austria_single_policy.ipynb`) 

**Answers**
- Answers to the exercises below can be found in another Jupyter Notebook in this folder called ``austria_multiple_policies-answers.ipynb``

**Introduction**

In this notebook, we investigate a number of different scenarios.
This process is streamlined with the utility function `solve_modified()`, used in a Python `with:` statement:

```python
with solve_modified(base_scenario, new_name) as scen:
   # (`scen` is automatically cloned from `base_scenario` with name `new_name`)
   
   # Your code to modify `scen` <---

   # (`scen` is automatically committed with modifications, then solved)

# Your code to investigate the solution of `scen` <---
```

All of the cloning, committing, and solving actions are handled for you.
Your job is to concentrate on identifying and updating the scenario variables and then investigating the results.

You can, of course, get the `Scenario` as you have worked on previously as well.

## Setup and Helper Variables

In [ ]:
import ixmp as ix

# launch the IX modeling platform using the local default database
mp = ix.Platform()

In [ ]:
country = "Austria"
horizon = range(2010, 2041, 10)

plants = [
    "bio_ppl",
    "coal_ppl",
    "gas_ppl",
    "oil_ppl",
    "hydro_ppl",
    "solar_pv_ppl",
    "wind_ppl",
]

lights = ["bulb", "cfl"]

In [ ]:
from message_ix.report import Reporter
from message_ix.testing import make_austria
from message_ix.util.tutorial import prepare_plots

# Create the baseline scenario and solve it
base_scen = make_austria(mp, solve=True)

# Create a reporter with tutorial plots
base_rep = Reporter.from_scenario(base_scen)
prepare_plots(base_rep)

# Wind Subsidies

Rerun the wind subsidy scenario using this framework.

In [ ]:
import numpy as np

from message_ix.util.tutorial import solve_modified

# Percent subsidy by year
subsidies = np.array([0.5, 0.5, 0.25, 0.0])

with solve_modified(base_scen, new_name="wind_subsidies") as wind_scen:
    # Load the investment cost data (cloned from `base_scen`) for wind_ppl
    data = wind_scen.par("inv_cost", filters=dict(technology="wind_ppl"))

    # Reduce the values according to the subsidy
    data["value"] = data["value"] * (1 - subsidies)

    # Overwrite the values in `wind_scen` at the same indices
    wind_scen.add_par("inv_cost", data)

# Display the new costs
data

In [ ]:
# Prepare a Reporter based on the solved scenario
wind_rep = Reporter.from_scenario(wind_scen)
prepare_plots(wind_rep)

# Run the same reporting on both the base and modified scenarios
for r in base_rep, wind_rep:
    r.set_filters(t=plants)
    r.get("plot new capacity")

# Demand-Side Learning

This model does not use `cfl`s in the baseline because they are too expensive. What happens if their cost reduces with time?

In [ ]:
# Cost as a fraction of the baseline cost, due to learning
learning = np.array([1.0, 0.6, 0.3, 0.1])

with solve_modified(base_scen, new_name="cheap_cfls") as cfl_scen:
    # Load the investment cost data (cloned from `base_scen`) for cfl
    data = cfl_scen.par("inv_cost", filters=dict(technology="cfl"))

    # Reduce the values according to the learning curve
    data["value"] = data["value"] * learning

    # Overwrite the values in `cfl_scen` at the same indices
    cfl_scen.add_par("inv_cost", data)

# Display the new costs
data

In [ ]:
import matplotlib_inline
from matplotlib.pyplot import style

# Appearance of plots
%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")
style.use("ggplot")

# Prepare a Reporter based on the solved scenario
cfl_rep = Reporter.from_scenario(cfl_scen)
prepare_plots(cfl_rep)

# Run the same reporting on both the base and modified scenarios
for r in base_rep, cfl_rep:
    r.set_filters(t=lights)
    r.get("plot new capacity")

# Exercise: Economic Assumptions

What is the effect of assuming a different interest rate? What if it is higher than the baseline? Lower? How does this affect prices?

In [ ]:
# Show the baseline interest rate
base_scen.par("interestrate")

In [ ]:
# Uncomment and complete the following code where "TODO" appears
# with solve_modified(base_scen, new_name="TODO") as econ_scen:
#    # TODO modify the interest rate to a different value

In [ ]:
# TODO based on the examples above, plot (a) new capacity and (b) prices

# Exercise: Carbon Tax

What effect does a carbon tax have on the system? What if it is phased in over time? What is the effect on energy prices?

Hints:

- Which emissions parameters are available from `scenario.par_list()`?
- Find out which fields are required using `scenario.idx_names(par_name)`
- Carbon taxes are normally provided in units of USD/tCO2
- A normal proposed carbon tax is ~30 USD/tCO2

In [ ]:
mp.close_db()